![imagenes](logo.png)

# Convirtiendo a formato tidy

Hasta ahora, solo has visto dataframes que ya estaban en formato "ordenado". Sin embargo, este no es siempre el caso con todos los dataframes del mundo. Si tu dataframe original está en formato ancho (no "ordenado") y deseas utilizar los paquetes ``ggplot2`` o ``dplyr``, primero deberás convertirlo al formato "ordenado". Para hacerlo, recomendamos usar la función ``pivot_longer()`` en el paquete ``tidyr`` de ``tidyverse``.

Volviendo a nuestro dataframe ``drinks_smaller`` de antes:

In [ ]:
library(fivethirtyeight)
library(tidyverse)

drinks_smaller <- drinks %>%
    filter(country %in% c("USA","China","Italy","Saudi Arabia","Mexico")) %>%
    select(-total_litres_of_pure_alcohol) %>%
    rename(beer = beer_servings, spirit = spirit_servings, wine = wine_servings)

drinks_smaller

Lo convertimos a formato "ordenado" usando la función ``pivot_longer()`` de la siguiente manera:

In [ ]:
drinks_smaller_tidy <- drinks_smaller %>% 
  pivot_longer(names_to = "type", 
               values_to = "servings", 
               cols = -country)

drinks_smaller_tidy

Establecemos los argumentos para ``pivot_longer()`` de la siguiente manera:

* ``names_to`` aquí corresponde al nombre de la variable en el nuevo marco de datos "ordenado"/largo que contendrá los nombres de las columnas de los datos originales. Observe cómo establecemos ``names_to = "type"``. En el ``drinks_smaller_tidy`` resultante, la columna ``type`` contiene los tres tipos de bebidas. Dado que el ``type`` es un nombre de variable que no aparece en ``drinks_smaller``, usamos comillas a su alrededor. Recibirás un error si solo escribes ``names_to = type``.


* ``values_to`` aquí es el nombre de la variable en el nuevo marco de datos "ordenado" que contendrá los valores de los datos originales. Observe cómo establecemos ``values_to = "servings"`` ya que cada uno de los valores numéricos en cada una de las columnas de cerveza, vino y licores de los datos de ``drinks_smaller`` corresponde a un valor de ``servings``. En ``drinks_smaller_tidy``, la columna ``servings`` contiene los 5 x 3 = 15 valores numéricos. Ten en cuenta nuevamente que la variable ``servings`` no aparece como una variable en ``drinks_smaller``, por lo que nuevamente necesita comillas para el argumento ``valuest_to``.

* El tercer argumento ``cols`` son las columnas en el marco de datos ``drinks_smaller`` que desea o no desea "ordenar". Observa cómo establecemos esto en ``-country``, lo que indica que no queremos "ordenar" la variable de país de ``drinks_smaller`` y más bien solo cerveza, licor y vino. Dado que el país es una columna que aparece en ``drinks_smaller``, no le ponemos comillas.

El tercer argumento (``cols``) está un poco matizado, así que consideremos un código que está escrito de forma ligeramente diferente pero que produce el mismo resultado:

In [ ]:
drinks_smaller %>% 
  pivot_longer(names_to = "type", 
               values_to = "servings", 
               cols = c(beer, spirit, wine))

Nota que el argumento ``cols`` ahora especifica qué columnas queremos "ordenar" con ``c(beer, spirit, wine)``, en lugar de las columnas que no queremos "ordenar" usando ``-country``. Usamos la función ``c()`` para crear un vector de las columnas de ``drinks_smaller`` que nos gustaría "ordenar". Ten en cuenta que dado que estas tres columnas aparecen una tras otra en ``drinks_smaller``, también podríamos hacer lo siguiente para el argumento ``cols``:

In [ ]:
drinks_smaller %>% 
  pivot_longer(names_to = "type", 
               values_to = "servings", 
               cols = beer:wine)

Con nuestro dataframe ``drinks_smaller_tidy`` ya en formato "tidy", podemos producir el diagrama de barras que vimos en la sección anterior usando ``geom_col()``. Observa que usamos ``geom_col()`` y no ``geom_bar()``, ya que nos gustaría mapear la variable de porciones "precontadas" a la estética y de las barras.

In [ ]:
ggplot(data = drinks_smaller_tidy, aes(x = country)) + 
    geom_col(aes(y = servings, fill = type),position = "dodge")

La conversión de datos de formato "ancho" a formato "ordenado" a menudo confunde a los nuevos usuarios de **R**. La única forma de aprender a familiarizarse con la función ``pivot_longer()`` es con práctica, práctica y más práctica utilizando diferentes conjuntos de datos. Por ejemplo, ejecute ``?pivot_longer`` y observa los ejemplos en la parte inferior del archivo de ayuda. 

Sin embargo, si deseas convertir un marco de datos "ordenado" a un formato "ancho", deberás usar la función ``pivot_wider()`` en su lugar. Ejecuta ``?pivot_wider`` y mira los ejemplos en la parte inferior del archivo de ayuda para ver ejemplos.

También puedes ver ejemplos de ``pivot_longer()`` y ``pivot_wider()`` en la página web [tidyverse.org](https://tidyr.tidyverse.org/dev/articles/pivot.html#pew). Hay un buen ejemplo para ver las diferentes funciones disponibles para ordenar datos y un estudio de caso usando datos de la Organización Mundial de la Salud en esa página web. Además, cada semana, la comunidad de aprendizaje en línea de R4DS publica un conjunto de datos en el evento semanal [#TidyTuesday](https://github.com/rfordatascience/tidytuesday) que puede servirte como un buen lugar para encontrar otros datos para explorar y transformar.

---
---

**Ejercicios**

**4.3.1.** Eche un vistazo al dataframe ``airline_safety`` incluido en el paquete ``fivethirtyeight``. Ejecuta lo siguiente:

In [ ]:
airline_safety

Después de leer el archivo de ayuda ejecutando ``?airline_safety``, vemos que ``airline_safety`` es un dataframe que contiene información sobre los registros de seguridad de diferentes compañías aéreas. Estos datos se informaron originalmente en el sitio web de periodismo de datos FiveThirtyEight.com en el artículo de Nate Silver, ["Should Travelers Avoid Flying Airlines That Have Had Crashes in the Past?”](https://fivethirtyeight.com/features/should-travelers-avoid-flying-airlines-that-have-had-crashes-in-the-past/). Consideremos solo las variables aerolíneas ``airlines`` y las relativas a total de muertes (``fatalities_85_89`` y ``fatalities_00_14``):

In [ ]:
airline_safety_smaller <- airline_safety %>%
    select(airline,starts_with("fatalities"))

airline_safety_smaller

Este dataframe no está en formato "ordenado". ¿Cómo convertirías este datafrmae para que esté en formato "ordenado"? En particular para que tenga una variable ``fatalities_years`` que indique el año del incidente y un recuento variable de los recuentos de fatalidades. Es decir, obtener la siguiente tabla de abajo. Observa que cada aerolínea aparece tres veces: fatalidades en 85-89, fatalidades de 00-14 y el total de fatalidades; además nota también el orden en que aparecen (primero total, luego 85-89 y finalmente 00-14)

**Sugerencia:** te puede convenir hacer un mutate primero para calcular los totales y luego aplicar el formato tidy. Luego, agrupa y a cada grupo añádele una columna de índices 1,2,3. Luego ordena cada grupo respecto de ese índice, elimina los grupos con ``ungroup()`` y selecciona las columnas de tu interés.

|airline                   |fatalities_type  | fatalities_year|
|:-------------------------|:----------------|---------------:|
|Aer Lingus                |fatalities_total |               0|
|Aer Lingus                |fatalities_00_14 |               0|
|Aer Lingus                |fatalities_85_99 |               0|
|Aeroflot                  |fatalities_total |             216|
|Aeroflot                  |fatalities_00_14 |              88|
|Aeroflot                  |fatalities_85_99 |             128|
|Aerolineas Argentinas     |fatalities_total |               0|
|Aerolineas Argentinas     |fatalities_00_14 |               0|
|Aerolineas Argentinas     |fatalities_85_99 |               0|
|Aeromexico                |fatalities_total |              64|
|Aeromexico                |fatalities_00_14 |               0|
|Aeromexico                |fatalities_85_99 |              64|
|Air Canada                |fatalities_total |               0|
|Air Canada                |fatalities_00_14 |               0|
|Air Canada                |fatalities_85_99 |               0|
|Air France                |fatalities_total |             416|
|Air France                |fatalities_00_14 |             337|
|Air France                |fatalities_85_99 |              79|
|Air India                 |fatalities_total |             487|
|Air India                 |fatalities_00_14 |             158|
|Air India                 |fatalities_85_99 |             329|
|Air New Zealand           |fatalities_total |               7|
|Air New Zealand           |fatalities_00_14 |               7|
|Air New Zealand           |fatalities_85_99 |               0|
|Alaska Airlines           |fatalities_total |              88|
|Alaska Airlines           |fatalities_00_14 |              88|
|Alaska Airlines           |fatalities_85_99 |               0|
|Alitalia                  |fatalities_total |              50|
|Alitalia                  |fatalities_00_14 |               0|
|Alitalia                  |fatalities_85_99 |              50|
|All Nippon Airways        |fatalities_total |               1|
|All Nippon Airways        |fatalities_00_14 |               0|
|All Nippon Airways        |fatalities_85_99 |               1|
|American                  |fatalities_total |             517|
|American                  |fatalities_00_14 |             416|
|American                  |fatalities_85_99 |             101|
|Austrian Airlines         |fatalities_total |               0|
|Austrian Airlines         |fatalities_00_14 |               0|
|Austrian Airlines         |fatalities_85_99 |               0|
|Avianca                   |fatalities_total |             323|
|Avianca                   |fatalities_00_14 |               0|
|Avianca                   |fatalities_85_99 |             323|
|British Airways           |fatalities_total |               0|
|British Airways           |fatalities_00_14 |               0|
|British Airways           |fatalities_85_99 |               0|
|Cathay Pacific            |fatalities_total |               0|
|Cathay Pacific            |fatalities_00_14 |               0|
|Cathay Pacific            |fatalities_85_99 |               0|
|China Airlines            |fatalities_total |             760|
|China Airlines            |fatalities_00_14 |             225|
|China Airlines            |fatalities_85_99 |             535|
|Condor                    |fatalities_total |              16|
|Condor                    |fatalities_00_14 |               0|
|Condor                    |fatalities_85_99 |              16|
|COPA                      |fatalities_total |              47|
|COPA                      |fatalities_00_14 |               0|
|COPA                      |fatalities_85_99 |              47|
|Delta / Northwest         |fatalities_total |             458|
|Delta / Northwest         |fatalities_00_14 |              51|
|Delta / Northwest         |fatalities_85_99 |             407|
|Egyptair                  |fatalities_total |             296|
|Egyptair                  |fatalities_00_14 |              14|
|Egyptair                  |fatalities_85_99 |             282|
|El Al                     |fatalities_total |               4|
|El Al                     |fatalities_00_14 |               0|
|El Al                     |fatalities_85_99 |               4|
|Ethiopian Airlines        |fatalities_total |             259|
|Ethiopian Airlines        |fatalities_00_14 |              92|
|Ethiopian Airlines        |fatalities_85_99 |             167|
|Finnair                   |fatalities_total |               0|
|Finnair                   |fatalities_00_14 |               0|
|Finnair                   |fatalities_85_99 |               0|
|Garuda Indonesia          |fatalities_total |             282|
|Garuda Indonesia          |fatalities_00_14 |              22|
|Garuda Indonesia          |fatalities_85_99 |             260|
|Gulf Air                  |fatalities_total |             143|
|Gulf Air                  |fatalities_00_14 |             143|
|Gulf Air                  |fatalities_85_99 |               0|
|Hawaiian Airlines         |fatalities_total |               0|
|Hawaiian Airlines         |fatalities_00_14 |               0|
|Hawaiian Airlines         |fatalities_85_99 |               0|
|Iberia                    |fatalities_total |             148|
|Iberia                    |fatalities_00_14 |               0|
|Iberia                    |fatalities_85_99 |             148|
|Japan Airlines            |fatalities_total |             520|
|Japan Airlines            |fatalities_00_14 |               0|
|Japan Airlines            |fatalities_85_99 |             520|
|Kenya Airways             |fatalities_total |             283|
|Kenya Airways             |fatalities_00_14 |             283|
|Kenya Airways             |fatalities_85_99 |               0|
|KLM                       |fatalities_total |               3|
|KLM                       |fatalities_00_14 |               0|
|KLM                       |fatalities_85_99 |               3|
|Korean Air                |fatalities_total |             425|
|Korean Air                |fatalities_00_14 |               0|
|Korean Air                |fatalities_85_99 |             425|
|LAN Airlines              |fatalities_total |              21|
|LAN Airlines              |fatalities_00_14 |               0|
|LAN Airlines              |fatalities_85_99 |              21|
|Lufthansa                 |fatalities_total |               2|
|Lufthansa                 |fatalities_00_14 |               0|
|Lufthansa                 |fatalities_85_99 |               2|
|Malaysia Airlines         |fatalities_total |             571|
|Malaysia Airlines         |fatalities_00_14 |             537|
|Malaysia Airlines         |fatalities_85_99 |              34|
|Pakistan International    |fatalities_total |             280|
|Pakistan International    |fatalities_00_14 |              46|
|Pakistan International    |fatalities_85_99 |             234|
|Philippine Airlines       |fatalities_total |              75|
|Philippine Airlines       |fatalities_00_14 |               1|
|Philippine Airlines       |fatalities_85_99 |              74|
|Qantas                    |fatalities_total |               0|
|Qantas                    |fatalities_00_14 |               0|
|Qantas                    |fatalities_85_99 |               0|
|Royal Air Maroc           |fatalities_total |              51|
|Royal Air Maroc           |fatalities_00_14 |               0|
|Royal Air Maroc           |fatalities_85_99 |              51|
|SAS                       |fatalities_total |             110|
|SAS                       |fatalities_00_14 |             110|
|SAS                       |fatalities_85_99 |               0|
|Saudi Arabian             |fatalities_total |             313|
|Saudi Arabian             |fatalities_00_14 |               0|
|Saudi Arabian             |fatalities_85_99 |             313|
|Singapore Airlines        |fatalities_total |              89|
|Singapore Airlines        |fatalities_00_14 |              83|
|Singapore Airlines        |fatalities_85_99 |               6|
|South African             |fatalities_total |             159|
|South African             |fatalities_00_14 |               0|
|South African             |fatalities_85_99 |             159|
|Southwest Airlines        |fatalities_total |               0|
|Southwest Airlines        |fatalities_00_14 |               0|
|Southwest Airlines        |fatalities_85_99 |               0|
|Sri Lankan / AirLanka     |fatalities_total |              14|
|Sri Lankan / AirLanka     |fatalities_00_14 |               0|
|Sri Lankan / AirLanka     |fatalities_85_99 |              14|
|SWISS                     |fatalities_total |             229|
|SWISS                     |fatalities_00_14 |               0|
|SWISS                     |fatalities_85_99 |             229|
|TACA                      |fatalities_total |               6|
|TACA                      |fatalities_00_14 |               3|
|TACA                      |fatalities_85_99 |               3|
|TAM                       |fatalities_total |             286|
|TAM                       |fatalities_00_14 |             188|
|TAM                       |fatalities_85_99 |              98|
|TAP - Air Portugal        |fatalities_total |               0|
|TAP - Air Portugal        |fatalities_00_14 |               0|
|TAP - Air Portugal        |fatalities_85_99 |               0|
|Thai Airways              |fatalities_total |             309|
|Thai Airways              |fatalities_00_14 |               1|
|Thai Airways              |fatalities_85_99 |             308|
|Turkish Airlines          |fatalities_total |             148|
|Turkish Airlines          |fatalities_00_14 |              84|
|Turkish Airlines          |fatalities_85_99 |              64|
|United / Continental      |fatalities_total |             428|
|United / Continental      |fatalities_00_14 |             109|
|United / Continental      |fatalities_85_99 |             319|
|US Airways / America West |fatalities_total |             247|
|US Airways / America West |fatalities_00_14 |              23|
|US Airways / America West |fatalities_85_99 |             224|
|Vietnam Airlines          |fatalities_total |             171|
|Vietnam Airlines          |fatalities_00_14 |               0|
|Vietnam Airlines          |fatalities_85_99 |             171|
|Virgin Atlantic           |fatalities_total |               0|
|Virgin Atlantic           |fatalities_00_14 |               0|
|Virgin Atlantic           |fatalities_85_99 |               0|
|Xiamen Airlines           |fatalities_total |              82|
|Xiamen Airlines           |fatalities_00_14 |               0|
|Xiamen Airlines           |fatalities_85_99 |              82|